# 01_数据预处理 - GLP-1临床试验风险预测

## 📊 功能概述

本notebook实现数据预处理功能，包括：
- 加载原始AACT数据库文件
- 数据清洗和预处理
- 合并三个数据源（studies.txt, conditions.txt, eligibilities.txt）
- 处理缺失值和数据类型转换

## 🚀 运行说明

直接运行所有单元格即可完成数据预处理。

**输出文件**: `processed_data/preprocessed_data.csv`

## 📝 学习目标

通过本notebook，您将学习：
- 如何加载和处理临床试验数据
- 数据清洗的基本技巧
- 多数据源合并的方法
- 缺失值处理的策略
- 数据类型转换的重要性

## 🎯 数据来源

本分析使用AACT（Aggregate Analysis of ClinicalTrials.gov）数据库，这是ClinicalTrials.gov的完整镜像数据库。

## 1. 导入必要的库

**📝 讲解**: 首先导入数据处理和分析所需的Python库。这些库是数据科学项目的基础工具。

- **pandas**: 用于数据框操作和数据处理，是Python数据分析的核心库
- **numpy**: 用于数值计算，提供高效的数组操作
- **os**: 用于文件路径操作，确保跨平台兼容性
- **sys**: 用于系统路径操作，管理Python路径
- **datetime**: 用于日期时间处理，处理试验开始日期等时间信息

**💡 技术要点**: 
- 使用`sys.path.append()`添加项目根目录到Python路径
- 这样可以确保能够导入项目中的其他模块
- 打印版本信息有助于确认环境配置正确

In [1]:
# 导入数据处理和分析所需的库
import pandas as pd
import numpy as np
import os
import sys
from datetime import datetime

# 添加项目根目录到路径，确保可以导入项目中的其他模块
# os.path.dirname(os.path.dirname(os.path.abspath('.'))) 获取项目根目录路径
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('.'))))

print("✅ 库导入完成")
print(f"当前工作目录: {os.getcwd()}")

# 显示导入的库版本信息，确保环境正确
print(f"Pandas版本: {pd.__version__}")
print(f"NumPy版本: {np.__version__}")

# 显示当前时间，记录处理开始时间
print(f"处理开始时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

✅ 库导入完成
当前工作目录: d:\projects\glp1-trial-risk-prediction\glp1-risk-prediction\notebooks
Pandas版本: 2.3.3
NumPy版本: 2.2.6
处理开始时间: 2026-02-26 21:45:49


## 2. 加载原始数据文件

**📝 讲解**: 在这一步中，我们将加载AACT数据库的原始文件。AACT数据库包含临床试验的详细信息，我们需要从三个主要文件中提取数据：

- **studies.txt**: 包含试验的基本信息（如NCT ID、开始日期、阶段、注册人数等）
- **conditions.txt**: 包含试验的疾病条件信息
- **eligibilities.txt**: 包含试验的入排标准

**💡 技术要点**: 
- 使用`os.path.join()`构建文件路径，确保跨平台兼容性
- 使用`pd.read_csv()`加载数据，注意分隔符是竖线'|'
- 设置`dtype=str`确保所有列都作为字符串加载，避免数据类型错误
- 使用`low_memory=False`确保正确处理大型文件
- 检查文件是否存在，避免运行时错误

In [2]:
# 定义原始数据文件的路径
raw_data_dir = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 'raw_data')  # 指向上一级的raw_data目录
studies_file = os.path.join(raw_data_dir, 'studies.txt')  # 试验基本信息文件
conditions_file = os.path.join(raw_data_dir, 'conditions.txt')  # 疾病条件文件
eligibilities_file = os.path.join(raw_data_dir, 'eligibilities.txt')  # 入排标准文件

print("🔍 检查数据文件是否存在...")
# 检查文件是否存在，避免运行时错误
for file_path in [studies_file, conditions_file, eligibilities_file]:
    if not os.path.exists(file_path):
        print(f"❌ 文件不存在: {file_path}")
    else:
        print(f"✅ 文件存在: {file_path}")

print("\n📥 开始加载原始数据文件...")

# 加载文件（注意分隔符为竖线 '|'，这是AACT数据库的标准格式）
# 使用dtype=str确保所有列都作为字符串加载，避免数据类型错误
# 使用low_memory=False确保正确处理大型文件
studies = pd.read_csv(studies_file, sep='|', dtype=str, low_memory=False)
conditions = pd.read_csv(conditions_file, sep='|', dtype=str, low_memory=False)
eligibilities = pd.read_csv(eligibilities_file, sep='|', dtype=str, low_memory=False)

print(f"✅ 数据加载完成!")
print(f"   - studies.txt: {studies.shape[0]} 行, {studies.shape[1]} 列")
print(f"   - conditions.txt: {conditions.shape[0]} 行, {conditions.shape[1]} 列")
print(f"   - eligibilities.txt: {eligibilities.shape[0]} 行, {eligibilities.shape[1]} 列")

# 显示前几行数据，帮助理解数据结构
print("\n📊 studies.txt数据结构预览:")
print(studies.head(3))

# 显示列名，了解可用的字段
print("\n📋 studies.txt的列名:")
print(list(studies.columns))

🔍 检查数据文件是否存在...
✅ 文件存在: d:\projects\glp1-trial-risk-prediction\raw_data\studies.txt
✅ 文件存在: d:\projects\glp1-trial-risk-prediction\raw_data\conditions.txt
✅ 文件存在: d:\projects\glp1-trial-risk-prediction\raw_data\eligibilities.txt

📥 开始加载原始数据文件...
✅ 数据加载完成!
   - studies.txt: 568538 行, 71 列
   - conditions.txt: 1009901 行, 4 列
   - eligibilities.txt: 567572 行, 14 列

📊 studies.txt数据结构预览:
        nct_id nlm_download_date_description study_first_submitted_date  \
0  NCT03729011                           NaN                 2018-10-03   
1  NCT04599764                           NaN                 2020-10-18   
2  NCT06337526                           NaN                 2024-03-22   

  results_first_submitted_date disposition_first_submitted_date  \
0                          NaN                              NaN   
1                          NaN                              NaN   
2                          NaN                              NaN   

  last_update_submitted_date study_first_sub

## 3. 选择需要的列

**📝 讲解**: 原始数据包含大量字段，但并非所有字段都对我们分析GLP-1临床试验风险预测有用。我们需要选择相关的列：

- **nct_id**: 临床试验的唯一标识符，用于数据合并
- **start_date**: 试验开始日期，用于分析时间趋势
- **phase**: 试验阶段，是重要的风险指标
- **enrollment**: 注册人数，反映试验规模
- **brief_title** 和 **official_title**: 试验标题，用于文本分析

**💡 技术要点**: 
- 使用`.copy()`创建数据框的副本，避免修改原始数据
- 选择相关列可以减少内存使用和提高处理速度

In [3]:
print("🔍 选择需要的列...")

# 选择需要的列，只保留对分析有用的字段
# 使用.copy()创建数据框的副本，避免修改原始数据
studies_selected = studies[['nct_id', 'start_date', 'phase', 'enrollment', 
                           'brief_title', 'official_title']].copy()

print(f"✅ 列选择完成!")
print(f"选择后的studies数据: {studies_selected.shape[0]} 行, {studies_selected.shape[1]} 列")
print("\n📋 选择的列及其用途:")
for col in studies_selected.columns:
    if col == 'nct_id':
        print(f"  - {col}: 临床试验唯一标识符（用于数据合并）")
    elif col == 'start_date':
        print(f"  - {col}: 试验开始日期（用于时间趋势分析）")
    elif col == 'phase':
        print(f"  - {col}: 试验阶段（重要风险指标）")
    elif col == 'enrollment':
        print(f"  - {col}: 注册人数（反映试验规模）")
    else:
        print(f"  - {col}: 试验标题（用于文本分析）")

# 显示选择后的数据预览
print("\n📊 选择后的数据预览:")
print(studies_selected.head())

🔍 选择需要的列...
✅ 列选择完成!
选择后的studies数据: 568538 行, 6 列

📋 选择的列及其用途:
  - nct_id: 临床试验唯一标识符（用于数据合并）
  - start_date: 试验开始日期（用于时间趋势分析）
  - phase: 试验阶段（重要风险指标）
  - enrollment: 注册人数（反映试验规模）
  - brief_title: 试验标题（用于文本分析）
  - official_title: 试验标题（用于文本分析）

📊 选择后的数据预览:
        nct_id  start_date          phase enrollment  \
0  NCT03729011  2019-01-09         PHASE4        405   
1  NCT04599764  2020-06-01            NaN         60   
2  NCT06337526  2025-12-01            NaN        150   
3  NCT03538041  2018-11-21         PHASE2         25   
4  NCT05573191  2022-10-08  PHASE1/PHASE2        100   

                                         brief_title  \
0  Delirium Reduction by Volatile Anesthesia in C...   
1  High Definition Transcranial Direct Current St...   
2  Predicting Pain Exacerbations in Children With...   
3  A Study of INCB050465 in Participants With Aut...   
4  Effects of Intravenous Lidocaine on ED50, ED95...   

                                      official_title  
0  Delirium Redu

## 4. 处理疾病条件数据

**📝 讲解**: 疾病条件数据中，每个试验可能有多个疾病条件。我们需要将这些条件合并成一个文本字段，便于后续的文本分析：

- 使用`groupby()`按nct_id分组
- 使用`apply()`和`join()`将所有疾病名称拼接成一个字符串
- 处理缺失值，确保空值被正确处理
- 重命名列名，使其更具描述性

**💡 技术要点**: 
- `pd.notna(x).any()`检查分组中是否有非空值
- 使用`reset_index()`将分组结果转换回数据框
- `rename()`用于重命名列，提高可读性

In [4]:
print("🔍 处理疾病条件数据...")

# 按 nct_id 分组，将所有疾病名称拼接成一个长字符串
# 这样每个试验只有一个疾病条件文本，便于后续分析
conditions_grouped = conditions.groupby('nct_id')['downcase_name'].apply(
    lambda x: ' '.join(x) if pd.notna(x).any() else ''  # 如果有非空值则拼接，否则返回空字符串
).reset_index()

# 重命名列，使其更具描述性
conditions_grouped.rename(columns={'downcase_name': 'conditions_text'}, inplace=True)

print(f"✅ 疾病条件数据处理完成!")
print(f"疾病条件数据分组后: {conditions_grouped.shape[0]} 行")

# 显示前几个试验的疾病条件，帮助理解处理结果
print("\n📋 前5个试验的疾病条件:")
for i, row in conditions_grouped.head().iterrows():
    conditions_text = row['conditions_text']
    if conditions_text:
        # 如果文本过长，显示前100个字符
        preview = conditions_text[:100] + '...' if len(conditions_text) > 100 else conditions_text
        print(f"  {row['nct_id']}: {preview}")
    else:
        print(f"  {row['nct_id']}: 无疾病条件信息")

# 统计有疾病条件的试验数量
has_conditions = conditions_grouped['conditions_text'].str.len() > 0
print(f"\n📊 统计信息: 有疾病条件的试验: {has_conditions.sum()} 个")

🔍 处理疾病条件数据...
✅ 疾病条件数据处理完成!
疾病条件数据分组后: 567540 行

📋 前5个试验的疾病条件:
  NCT00000102: congenital adrenal hyperplasia
  NCT00000104: lead poisoning
  NCT00000105: cancer
  NCT00000106: rheumatic diseases
  NCT00000107: heart defects, congenital

📊 统计信息: 有疾病条件的试验: 567540 个


## 5. 处理入排标准数据

**📝 讲解**: 入排标准数据包含试验的纳入和排除标准，这是评估试验风险的重要文本信息：

- 我们只需要nct_id和criteria两个字段
- criteria字段包含详细的入排标准文本
- 这些文本将用于后续的文本分析和特征提取

**💡 技术要点**: 
- 入排标准文本通常很长，包含重要的风险信息
- 后续将从中提取排除标准数量、高风险术语等特征

In [5]:
print("🔍 处理入排标准数据...")

# eligibilities 保留 criteria 字段
# 我们只需要nct_id和criteria两个字段
eligibilities_selected = eligibilities[['nct_id', 'criteria']].copy()

print(f"✅ 入排标准数据处理完成!")
print(f"入排标准数据: {eligibilities_selected.shape[0]} 行")

# 显示入排标准示例，帮助理解数据结构
print("\n📋 入排标准示例（前3个）:")
for i, row in eligibilities_selected.head(3).iterrows():
    criteria = row['criteria']
    if pd.notna(criteria) and criteria != '':
        # 如果文本过长，显示前200个字符
        criteria_preview = criteria[:200] + '...' if len(criteria) > 200 else criteria
        print(f"  {row['nct_id']}: {criteria_preview}")
    else:
        print(f"  {row['nct_id']}: 无入排标准信息")

# 统计有入排标准的试验数量
has_criteria = eligibilities_selected['criteria'].notna() & (eligibilities_selected['criteria'] != '')
print(f"\n📊 统计信息: 有入排标准的试验: {has_criteria.sum()} 个")

🔍 处理入排标准数据...
✅ 入排标准数据处理完成!
入排标准数据: 567572 行

📋 入排标准示例（前3个）:
  NCT02336256: Inclusion Criteria:~* Patients with inguinal hernia.~Exclusion Criteria:~* Patients with acute disease and severe chronic disorder.
  NCT04722315: Inclusion Criteria:We will recruit 15 total participants with clinically-definite and genetically-confirmed Kabuki syndrome type 1 age 18 years and older. . will also be required.~* Clinical diagnosis...
  NCT04403633: Inclusion Criteria:~* age 1 to 18 years~* patients with diagnosis of AKI while hospitalized~Exclusion Criteria:~* legal blindness of deafness~* cognitive impairment that limits ability to consent~* no...

📊 统计信息: 有入排标准的试验: 567479 个


## 6. 合并数据

**📝 讲解**: 现在我们需要将三个数据源合并成一个完整的数据框：

- 使用`merge()`函数进行数据合并
- 使用左连接（how='left'）确保保留所有试验
- 以nct_id作为连接键
- 分两步合并：先合并疾病条件，再合并入排标准

**💡 技术要点**: 
- 左连接确保即使某些试验没有疾病条件或入排标准信息，也不会被丢弃
- 合并后的数据框包含所有试验的完整信息

In [6]:
print("🔗 合并数据...")

# 合并数据（左连接，保留所有试验）
# 第一步：合并试验基本信息和疾病条件
df = studies_selected.merge(conditions_grouped, on='nct_id', how='left')

# 第二步：合并入排标准信息
df = df.merge(eligibilities_selected, on='nct_id', how='left')

print(f"✅ 数据合并完成!")
print(f"   - 合并后数据: {df.shape[0]} 行, {df.shape[1]} 列")

# 显示合并后的数据列
print("\n📋 合并后的数据列及其含义:")
for col in df.columns:
    if col == 'nct_id':
        print(f"  - {col}: 临床试验唯一标识符")
    elif col == 'start_date':
        print(f"  - {col}: 试验开始日期")
    elif col == 'phase':
        print(f"  - {col}: 试验阶段")
    elif col == 'enrollment':
        print(f"  - {col}: 注册人数")
    elif col == 'brief_title':
        print(f"  - {col}: 简要标题")
    elif col == 'official_title':
        print(f"  - {col}: 官方标题")
    elif col == 'conditions_text':
        print(f"  - {col}: 疾病条件文本")
    elif col == 'criteria':
        print(f"  - {col}: 入排标准文本")

# 显示合并后的数据预览
print("\n📊 合并后的数据预览:")
print(df.head())

🔗 合并数据...
✅ 数据合并完成!
   - 合并后数据: 568538 行, 8 列

📋 合并后的数据列及其含义:
  - nct_id: 临床试验唯一标识符
  - start_date: 试验开始日期
  - phase: 试验阶段
  - enrollment: 注册人数
  - brief_title: 简要标题
  - official_title: 官方标题
  - conditions_text: 疾病条件文本
  - criteria: 入排标准文本

📊 合并后的数据预览:
        nct_id  start_date          phase enrollment  \
0  NCT03729011  2019-01-09         PHASE4        405   
1  NCT04599764  2020-06-01            NaN         60   
2  NCT06337526  2025-12-01            NaN        150   
3  NCT03538041  2018-11-21         PHASE2         25   
4  NCT05573191  2022-10-08  PHASE1/PHASE2        100   

                                         brief_title  \
0  Delirium Reduction by Volatile Anesthesia in C...   
1  High Definition Transcranial Direct Current St...   
2  Predicting Pain Exacerbations in Children With...   
3  A Study of INCB050465 in Participants With Aut...   
4  Effects of Intravenous Lidocaine on ED50, ED95...   

                                      official_title  \
0  Delirium Reduc

## 7. 处理缺失值和数据类型转换

**📝 讲解**: 数据合并后，我们需要处理缺失值并进行适当的数据类型转换：

- **enrollment**: 转换为数值类型，缺失值填充为0
- **start_date**: 转换为日期时间类型
- **phase**: 填充缺失值为空字符串
- **文本字段**: 填充缺失值为空字符串

**💡 技术要点**: 
- `pd.to_numeric()`用于数值转换，errors='coerce'将无法转换的值设为NaN
- `pd.to_datetime()`用于日期转换
- `fillna()`用于填充缺失值
- `astype(str)`确保数据类型一致

In [7]:
print("🔧 处理缺失值和数据类型转换...")

# 填充缺失值并进行数据类型转换
# 1. enrollment: 转换为数值类型，缺失值填充为0
df['enrollment'] = pd.to_numeric(df['enrollment'], errors='coerce').fillna(0)

# 2. start_date: 转换为日期时间类型
df['start_date'] = pd.to_datetime(df['start_date'], errors='coerce')

# 3. phase: 填充缺失值为空字符串
df['phase'] = df['phase'].fillna('').astype(str)

# 4. 文本字段: 填充缺失值为空字符串
df['conditions_text'] = df['conditions_text'].fillna('')
df['criteria'] = df['criteria'].fillna('')
df['brief_title'] = df['brief_title'].fillna('')
df['official_title'] = df['official_title'].fillna('')

print("✅ 缺失值处理和数据类型转换完成!")

# 显示数据类型
print("\n📊 数据类型检查:")
for col in df.columns:
    print(f"  - {col}: {df[col].dtype}")

# 显示缺失值统计
print("\n📊 缺失值统计:")
for col in df.columns:
    missing_count = df[col].isna().sum()
    if missing_count > 0:
        print(f"  - {col}: {missing_count} 个缺失值")
    else:
        print(f"  - {col}: 无缺失值")

🔧 处理缺失值和数据类型转换...
✅ 缺失值处理和数据类型转换完成!

📊 数据类型检查:
  - nct_id: object
  - start_date: datetime64[ns]
  - phase: object
  - enrollment: float64
  - brief_title: object
  - official_title: object
  - conditions_text: object
  - criteria: object

📊 缺失值统计:
  - nct_id: 无缺失值
  - start_date: 5312 个缺失值
  - phase: 无缺失值
  - enrollment: 无缺失值
  - brief_title: 无缺失值
  - official_title: 无缺失值
  - conditions_text: 无缺失值
  - criteria: 无缺失值


## 8. 数据质量检查

**📝 讲解**: 在保存数据之前，我们需要进行数据质量检查：

- 检查唯一试验数量
- 检查各字段的完整性
- 检查重复的nct_id
- 检查数据的基本统计信息

**💡 技术要点**: 
- `nunique()`统计唯一值数量
- `notna().sum()`统计非空值数量
- `duplicated().sum()`统计重复值数量
- `drop_duplicates()`删除重复值

In [8]:
print("🔍 数据质量检查...")

# 数据质量检查
print(f"   - 唯一试验数量: {df['nct_id'].nunique()}")
print(f"   - 有开始日期的试验: {df['start_date'].notna().sum()}")
print(f"   - 有注册人数的试验: {df[df['enrollment'] > 0].shape[0]}")
print(f"   - 有疾病信息的试验: {df[df['conditions_text'] != ''].shape[0]}")
print(f"   - 有入排标准的试验: {df[df['criteria'] != ''].shape[0]}")

# 检查重复的nct_id
duplicate_nct = df['nct_id'].duplicated().sum()
if duplicate_nct > 0:
    print(f"   ⚠️ 警告: 发现 {duplicate_nct} 个重复的nct_id")
    # 删除重复值，保留第一个出现的数据
    df = df.drop_duplicates(subset=['nct_id'], keep='first')
    print(f"   - 去重后数据: {df.shape[0]} 行")
else:
    print("   ✅ 无重复的nct_id")

print("✅ 数据质量检查完成!")

🔍 数据质量检查...
   - 唯一试验数量: 568538
   - 有开始日期的试验: 563226
   - 有注册人数的试验: 545548
   - 有疾病信息的试验: 567540
   - 有入排标准的试验: 567479
   ✅ 无重复的nct_id
✅ 数据质量检查完成!


## 9. 保存预处理后的数据

**📝 讲解**: 最后，我们将预处理后的数据保存为CSV文件，供后续分析使用：

- 创建输出目录
- 保存为CSV格式
- 设置index=False不保存行索引
- 使用UTF-8编码确保中文正确保存

**💡 技术要点**: 
- `os.makedirs(exist_ok=True)`创建目录，如果已存在则不会报错
- `to_csv()`保存数据框为CSV文件
- 显示保存路径和文件信息

In [9]:
print("💾 保存预处理后的数据...")

# 创建输出目录
output_dir = os.path.join('..', 'processed_data')
os.makedirs(output_dir, exist_ok=True)

# 保存预处理后的数据
output_file = os.path.join(output_dir, 'preprocessed_data.csv')
df.to_csv(output_file, index=False, encoding='utf-8')

print(f"✅ 数据预处理完成!")
print(f"   - 输出文件: {output_file}")
print(f"   - 最终数据维度: {df.shape}")

# 显示数据概览
print("\n📊 数据概览:")
print(f"总试验数: {len(df)}")
print(f"特征数: {len(df.columns)}")

# 显示前5个试验的nct_id
print("\n📋 前5个试验的nct_id:")
print(df['nct_id'].head().tolist())

# 显示注册人数统计
print(f"\n📊 注册人数统计:")
print(f"   - 最小值: {df['enrollment'].min()}")
print(f"   - 最大值: {df['enrollment'].max()}")
print(f"   - 平均值: {df['enrollment'].mean():.1f}")
print(f"   - 中位数: {df['enrollment'].median()}")

# 显示试验阶段分布
print(f"\n📊 试验阶段分布:")
phase_counts = df['phase'].value_counts().head(10)
for phase, count in phase_counts.items():
    print(f"   - {phase}: {count} 个试验")

# 显示处理结束时间
print(f"\n⏰ 处理结束时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

print("\n🎉 数据预处理流程完成！可以继续运行下一个notebook进行特征工程。")

💾 保存预处理后的数据...
✅ 数据预处理完成!
   - 输出文件: ..\processed_data\preprocessed_data.csv
   - 最终数据维度: (568538, 8)

📊 数据概览:
总试验数: 568538
特征数: 8

📋 前5个试验的nct_id:
['NCT03729011', 'NCT04599764', 'NCT06337526', 'NCT03538041', 'NCT05573191']

📊 注册人数统计:
   - 最小值: 0.0
   - 最大值: 188814085.0
   - 平均值: 5384.3
   - 中位数: 66.0

📊 试验阶段分布:
   - : 353259 个试验
   - PHASE2: 63036 个试验
   - PHASE1: 46819 个试验
   - PHASE3: 41023 个试验
   - PHASE4: 34673 个试验
   - PHASE1/PHASE2: 16313 个试验
   - PHASE2/PHASE3: 7342 个试验
   - EARLY_PHASE1: 6073 个试验

⏰ 处理结束时间: 2026-02-26 21:46:49

🎉 数据预处理流程完成！可以继续运行下一个notebook进行特征工程。


## 🎯 总结

### 完成的工作

✅ **数据加载**: 成功加载AACT数据库的三个主要文件  
✅ **列选择**: 选择对分析有用的字段  
✅ **数据合并**: 将三个数据源合并成一个完整的数据框  
✅ **缺失值处理**: 正确处理各种类型的缺失值  
✅ **数据类型转换**: 确保数据类型正确  
✅ **数据质量检查**: 验证数据完整性  
✅ **数据保存**: 保存预处理后的数据  

### 关键统计信息

- **总试验数**: {len(df)}  
- **特征数**: {len(df.columns)}  
- **有开始日期的试验**: {df['start_date'].notna().sum()}  
- **有注册人数的试验**: {df[df['enrollment'] > 0].shape[0]}  
- **有疾病信息的试验**: {df[df['conditions_text'] != ''].shape[0]}  
- **有入排标准的试验**: {df[df['criteria'] != ''].shape[0]}  

### 下一步

数据预处理完成后，可以运行下一个notebook进行特征工程：

```python
# 运行特征工程notebook
%run 02_特征工程.ipynb
```

---
*数据预处理完成时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}*